# Projet Python : Santé

## Récupération des données

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [76]:
#test avec Selenium

In [77]:
#pip install selenium

     |████████████████████████████████| 904 kB 4.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import time  #mettre des script en pause pour les chargement de la page
from selenium import webdriver #module qui gère le navigateur
from selenium.webdriver.common.keys import Keys #la classe keus permet d'utiliser les touches du clavier
from selenium.webdriver import Chrome

In [2]:
#path_to_web_driver = "chromedriver"

In [3]:
#chrome_options=webdriver.ChromeOptions()

In [88]:

#On crée une instance du web-driver Chrome et on va sur la page sélectionnée
driver = webdriver.Chrome("/Users/antoineloubeyre/Downloads/chromedriver-5")

driver.get("https://geodes.santepubliquefrance.fr/#view=map2&c=indicator")

time.sleep(2)

In [90]:
# On recupere la bar de recherche
search_bar = driver.find_element_by_name("search")
search_bar.send_keys("covid")
search_bar.send_keys(Keys.ENTER)

time.sleep(1)

# Clique sur le sous lien Covid
driver.find_element_by_class_name("ui-collapsible-content ").click()

time.sleep(1)


#Clique sur les données hospitalières
driver.find_element_by_xpath("//span[@class='ui-li-label level2']").click()


time.sleep(1)


#Sélection nombre cumulé de décès par âge
driver.find_element_by_xpath("//h3[@class='ui-collapsible-heading filter1']").click()

time.sleep(1)


#Sélection tous les âges
driver.find_element_by_xpath("//li[@data-jsv='#198^#506_#507_']").click()



In [107]:
# Nouvelle recherche pour Asthme

search_bar.clear()

time.sleep(1)
search_bar.send_keys("Asthme")
search_bar.send_keys(Keys.ENTER)

time.sleep(1)



In [108]:
#Sélection tous les âges 

#pb prends seulement 65 ans et plus

driver.find_element_by_xpath("//li[@data-jsv='#1314^#2149_#2150_']").click()




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@data-jsv='#1314^#2149_#2150_']"}
  (Session info: chrome=86.0.4240.183)


In [69]:
# Nouvelle recherche pour Allergies

search_bar.clear()

time.sleep(1)
search_bar.send_keys("Allergie")
search_bar.send_keys(Keys.ENTER)


timple.sleep(1)

#sélection tous les âges
driver.find_element_by_xpath("//li[@data-jsv='#2878^#4002_#4003_']").click()




In [35]:
#Sélection options régions
driver.find_element_by_xpath("//select[@id='tm_view_cb']").click()

#sleep(2)

#Sélection des départements 2019
driver.find_element_by_xpath("//option[@data-jsv='/387_#388_']").click()

In [12]:
# Affichage des données sur la carte

driver.find_element_by_xpath("//li[@data-jsv='/1000_']").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//li[@data-jsv='/1000_']"}
  (Session info: chrome=86.0.4240.183)


In [14]:
driver.find_element_by_xpath("//li[@class='js-tablist__item tiny-hiddenXX tabs__item']").click()

In [15]:
#Récupération du tableau

results=driver.find_element_by_xpath("//table[@id='tm_datatable']").text

In [16]:
results=driver.find_element_by_xpath("//div[@class='dataTables_scroll']").text

In [327]:
print(results)
type(results)

Code Libellé Nombre quotidien de nouveaux décès 2020-11-04
01 Ain 6
02 Aisne 5
03 Allier 7
04 Alpes-de-Haute-Provence 0
05 Hautes-Alpes 0
06 Alpes-Maritimes 2
07 Ardèche 5
08 Ardennes 0
09 Ariège 0
10 Aube 0
11 Aude 2
12 Aveyron 0
13 Bouches-du-Rhône 26
14 Calvados 3
15 Cantal 0
16 Charente 0
17 Charente-Maritime 2
18 Cher 2
19 Corrèze 0
21 Côte-d'Or 5
22 Côtes-d'Armor 1
23 Creuse 0
24 Dordogne 1
25 Doubs 2
26 Drôme 8
27 Eure 4
28 Eure-et-Loir 3
29 Finistère 0
2A Corse-du-Sud 1
2B Haute-Corse 1
30 Gard 6
31 Haute-Garonne 7
32 Gers 1
33 Gironde 4
34 Hérault 3
35 Ille-et-Vilaine 2
36 Indre 0
37 Indre-et-Loire 1
38 Isère 19
39 Jura 1
40 Landes 0
41 Loir-et-Cher 1
42 Loire 11
43 Haute-Loire 2
44 Loire-Atlantique 4
45 Loiret 1
46 Lot 0
47 Lot-et-Garonne 0
48 Lozère 3
49 Maine-et-Loire 6
50 Manche 2
51 Marne 1
52 Haute-Marne 2
53 Mayenne 5
54 Meurthe-et-Moselle 3
55 Meuse 1
56 Morbihan 2
57 Moselle 2
58 Nièvre 1
59 Nord 24
60 Oise 4
61 Orne 2
62 Pas-de-Calais 9
63 Puy-de-Dôme 9
64 Pyrénées-A

str

In [17]:
import io
import pandas as pd

data = io.StringIO(results)
df = pd.read_csv(data, sep=";")

In [341]:
df.head(10)
#df.columns


,Code Libellé Nombre quotidien de nouveaux décès 2020-11-04
0,01 Ain 6
1,02 Aisne 5
2,03 Allier 7
3,04 Alpes-de-Haute-Provence 0
4,05 Hautes-Alpes 0
5,06 Alpes-Maritimes 2
6,07 Ardèche 5
7,08 Ardennes 0
8,09 Ariège 0
9,10 Aube 0


In [18]:
df2=df['Code Libellé Nombre quotidien de nouveaux décès 2020-11-04'].str.split(" ",1)

KeyError: 'Code Libellé Nombre quotidien de nouveaux décès 2020-11-04'

In [337]:
type(df2)
df2.head()


0                        [01, Ain 6]
1                      [02, Aisne 5]
2                     [03, Allier 7]
3    [04, Alpes-de-Haute-Provence 0]
4               [05, Hautes-Alpes 0]
Name: Code Libellé Nombre quotidien de nouveaux décès 2020-11-04, dtype: object

In [342]:
out = pd.DataFrame(df["Code Libellé Nombre quotidien de nouveaux décès 2020-11-04"].str.split(' ',2).tolist(),columns=['Code','departement','décès'])
out.drop(index=0,inplace=True)

In [289]:
out.head(30)

,Code,departement,décès
1,02,Martinique,22
2,03,Guyane,0
3,04,Réunion,3
4,06,Mayotte,12
5,11,Île-de-France,3 941
6,24,Centre-Val,de Loire 348
7,27,Bourgogne,et Franche-Comté 461
8,28,Normandie,203
9,32,Hauts-de-France,459
10,44,Grand,Est 2 479


In [268]:
split_data = df["Code Libellé Nombre cumulé de décès par âge 2020-08-16 - tous âges"].str.split(", ")
data = split_data.to_list()
print(data)
#names = ["Code", "Departement","dead","autres"]
#new_df = pd.DataFrame(data, columns=names)

#print(new_df)

[['01 Guadeloupe 2'], ['02 Martinique 22'], ['03 Guyane 0'], ['04 Réunion 3'], ['06 Mayotte 12'], ['11 Île-de-France 3\u202f941'], ['24 Centre-Val de Loire 348'], ['27 Bourgogne et Franche-Comté 461'], ['28 Normandie 203'], ['32 Hauts-de-France 459'], ['44 Grand Est 2\u202f479'], ['52 Pays de la Loire 202'], ['53 Bretagne 122'], ['75 Nouvelle Aquitaine 473'], ['76 Occitanie 514'], ['84 Auvergne et Rhône-Alpes 1\u202f408'], ["93 Provence-Alpes-Côte d'Azur 691"], ['94 Corse 0'], ['975 Miquelon-Langlade et Saint Pierre N/A - résultat non disponible'], ['977 Saint-Barthélemy N/A - résultat non disponible'], ['978 Saint-Martin N/A - résultat non disponible']]
